<a href="https://colab.research.google.com/github/chenrufeng/JNoteBook/blob/master/MNIST%2BDensnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class pytorch_Net(nn.Module):
    def __init__(self):
        super(pytorch_Net, self).__init__()
        self.num_channels = 1
        self.image_size = 28
        self.num_labels = 10
        self.conv2d_1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2d_2 = nn.Conv2d(32, 32, 3, 1)
        self.conv2d_3 = nn.Conv2d(32, 64, 3, 1)
        self.conv2d_4 = nn.Conv2d(64, 64, 3, 1)

        self.dense_1 = nn.Linear(4 * 4 * 64, 200)
        self.dense_2 = nn.Linear(200, 200)
        self.dense_3 = nn.Linear(200, 10)


    def forward(self, x):
        x = F.relu(self.conv2d_1(x))
        x = F.relu(self.conv2d_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2d_3(x))
        x = F.relu(self.conv2d_4(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.permute((0, 2, 3, 1))

        x = x.contiguous().view(-1, 4 * 4 * 64)
        x = x.view(-1, 4 * 4 * 64)
        x = F.relu(self.dense_1(x))
        x = F.relu(self.dense_2(x))
        x = self.dense_3(x)
        return F.log_softmax(x, dim=1)

In [ ]:
class pytorch_keras_Net(nn.Module):
    def __init__(self):
        super(pytorch_keras_Net, self).__init__()
        self.num_channels = 1
        self.image_size = 28
        self.num_labels = 10
        self.conv2d_1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2d_2 = nn.Conv2d(32, 32, 3, 1)
        self.conv2d_3 = nn.Conv2d(32, 64, 3, 1)
        self.conv2d_4 = nn.Conv2d(64, 64, 3, 1)

        self.dense_1 = nn.Linear(4 * 4 * 64, 200)
        self.dense_2 = nn.Linear(200, 200)
        self.dense_3 = nn.Linear(200, 10)


    def forward(self, x):
        x = F.relu(self.conv2d_1(x))
        x = F.relu(self.conv2d_2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2d_3(x))
        x = F.relu(self.conv2d_4(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.permute((0, 2, 3, 1))

        x = x.contiguous().view(-1, 4 * 4 * 64)
        x = x.view(-1, 4 * 4 * 64)
        x = F.relu(self.dense_1(x))
        x = F.relu(self.dense_2(x))
        x = self.dense_3(x)
        return F.log_softmax(x, dim=1)

In [ ]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            # print("______data.shape:", data.shape)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
class args:
	load_keras= False
	batch_size=64
	test_batch_size=1000
	epochs=10
	lr=0.01
	momentum=0.5
	no_cuda=False
	seed=1
	log_interval=100
	save_model=True
print(args)
print(args.no_cuda)

<class '__main__.args'>
False


In [ ]:
use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
	datasets.MNIST('../data', train=True, download=True,
				   transform=transforms.Compose([
					   transforms.ToTensor(),
					   transforms.Normalize((0.1307,), (0.3081,))
				   ])),
	batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
	datasets.MNIST('../data', train=False, transform=transforms.Compose([
		transforms.ToTensor(),
		transforms.Normalize((0.1307,), (0.3081,))
	])),
	batch_size=args.test_batch_size, shuffle=True, **kwargs)
if args.load_keras:
	model = pytorch_keras_Net.to(device)
	model.load_state_dict(torch.load("pyt_model.pt"))
else:
	model = pytorch_Net().to(device)
# model.load_state_dict(torch.load("mnist_cnn.pt"))
for name, param in model.named_parameters():
	print("name:", name)
	print("param:", param.shape)


optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

for epoch in range(1, args.epochs + 1):
	train(args, model, device, train_loader, optimizer, epoch)
	test(args, model, device, test_loader)

if (args.save_model):
	torch.save(model.state_dict(), "mnist_cnn.pt")

name: conv2d_1.weight
param: torch.Size([32, 1, 3, 3])
name: conv2d_1.bias
param: torch.Size([32])
name: conv2d_2.weight
param: torch.Size([32, 32, 3, 3])
name: conv2d_2.bias
param: torch.Size([32])
name: conv2d_3.weight
param: torch.Size([64, 32, 3, 3])
name: conv2d_3.bias
param: torch.Size([64])
name: conv2d_4.weight
param: torch.Size([64, 64, 3, 3])
name: conv2d_4.bias
param: torch.Size([64])
name: dense_1.weight
param: torch.Size([200, 1024])
name: dense_1.bias
param: torch.Size([200])
name: dense_2.weight
param: torch.Size([200, 200])
name: dense_2.bias
param: torch.Size([200])
name: dense_3.weight
param: torch.Size([10, 200])
name: dense_3.bias
param: torch.Size([10])
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303075
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.299266
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.284276
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.086459
